In [ ]:
import sys  
sys.path.insert(0, 'D:/TU_KL/thesis/cnn_compression')

import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from scipy.fftpack import dct
from prototypes.DCT_layers import DCT_linear_layer, LinearDCT
import math
## cifar10 download problem solve
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='../data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='../data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
class AlexNet(nn.Module):

    def __init__(self, num_classes: int = 10) -> None:
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
#             DCT_conv_layer(3, 64, kernel_size=3, stride=2, padding=1),
#             nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),   
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
#             DCT_conv_layer(64, 192, kernel_size=3, padding=1),
#             nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),    
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
#             DCT_conv_layer(192, 384, kernel_size=3, padding=1),
#             nn.BatchNorm2d(384),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
#             DCT_conv_layer(384, 256, kernel_size=3, padding=1),
#             nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
#             DCT_conv_layer(256, 256, kernel_size=3, padding=1),
#             nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),

        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
#             nn.Linear(256 * 2 * 2, 4096),
            DCT_linear_layer(256 * 2 * 2,4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            DCT_linear_layer(4096,4096),
#             nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
#             DCT_layer(num_classes)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = x.view(x.size(0), 256 * 2 * 2)
        x = self.classifier(x)
        return x
    
net = AlexNet(num_classes=10).to(device)
print(net)


In [ ]:
def train(dataloader,model,criterion,optimizer):
#     torch.autograd.detect_anomaly() 
    train_loss = 0.0
    for X, y in dataloader:
        inputs, labels = X.to(device), y.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print statistics
        train_loss += loss.item()*inputs.size(0)
    train_loss = train_loss/len(dataloader)
    
    print(f'Training Loss: {train_loss:.8f}')

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum=0.9)

print("DCT_fc_AlexNet..........new linear_dct")
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, net, criterion, optimizer)
    test(testloader, net, criterion)


In [ ]:
## no. of paramaters
print('no of trainable params:', sum(p.numel() for p in net.parameters() if p.requires_grad))